In [19]:
from pathlib import Path
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.backends import default_backend
from cryptography import x509

In [20]:
def egcd(a, b):
    """Algorithme d'Euclide étendu.
    Retourne (g, x, y) tels que g = gcd(a, b) et ax + by = g.
    """
    if b == 0:
        return a, 1, 0
    g, x1, y1 = egcd(b, a % b)
    return g, y1, x1 - (a // b) * y1

In [21]:
def inv_mod(a, n):
    """Inverse modulaire : trouve x tel que a*x ≡ 1 mod n."""
    g, x, y = egcd(a, n)
    if g != 1:
        raise ValueError("Pas d'inverse modulaire")
    return x % n

In [22]:
def compute_u_i(N1, N2, N3):
    N = N1 * N2 * N3

    N_hat1 = N // N1
    N_hat2 = N // N2
    N_hat3 = N // N3

    u1 = inv_mod(N_hat1 % N1, N1)
    u2 = inv_mod(N_hat2 % N2, N2)
    u3 = inv_mod(N_hat3 % N3, N3)

    return (u1, u2, u3), (N_hat1, N_hat2, N_hat3)


In [23]:
def compute_m_cubed(c1, c2, c3, N1, N2, N3):
    (u1, u2, u3), (Nh1, Nh2, Nh3) = compute_u_i(N1, N2, N3)
    N = N1 * N2 * N3

    m3 = (c1 * u1 * Nh1 +
          c2 * u2 * Nh2 +
          c3 * u3 * Nh3) % N

    return m3, N


In [24]:
def integer_cube_root(n):
    """Renvoie la racine cubique entière si elle existe exactement."""
    x = int(round(n ** (1/3)))
    while (x + 1)**3 <= n:
        x += 1
    while x**3 > n:
        x -= 1
    if x**3 != n:
        raise ValueError("La racine cubique n'est pas un entier parfait.")
    return x


In [25]:
def recover_message_from_broadcast(c1, c2, c3, N1, N2, N3):
    m3, N = compute_m_cubed(c1, c2, c3, N1, N2, N3)
    m = integer_cube_root(m3)
    return m


In [26]:
import base64
from pathlib import Path

def read_cipher_as_int(filename):
    data_b64 = Path(filename).read_bytes().strip()
    data_bytes = base64.b64decode(data_b64)
    return int.from_bytes(data_bytes, "big")

c0 = read_cipher_as_int("c0")
c1 = read_cipher_as_int("c1")
c2 = read_cipher_as_int("c2")


In [27]:
def read_modulus_from_pem(filename: str):
    """
    Lit un fichier de clé/certificat RSA (PEM ou DER) et renvoie (N, e).
    Gère plusieurs cas :
      - -----BEGIN PUBLIC KEY-----
      - -----BEGIN RSA PUBLIC KEY-----
      - -----BEGIN CERTIFICATE-----
      - fichiers DER (clé ou certificat)
    """
    data = Path(filename).read_bytes()

    # Cas PEM certificat
    if b"BEGIN CERTIFICATE" in data:
        cert = x509.load_pem_x509_certificate(data, backend=default_backend())
        pubkey = cert.public_key()

    # Cas PEM clé publique
    elif b"BEGIN PUBLIC KEY" in data or b"BEGIN RSA PUBLIC KEY" in data:
        pubkey = serialization.load_pem_public_key(data, backend=default_backend())

    else:
        # On essaie d'abord DER clé publique, puis DER certificat
        try:
            pubkey = serialization.load_der_public_key(data, backend=default_backend())
        except Exception:
            cert = x509.load_der_x509_certificate(data, backend=default_backend())
            pubkey = cert.public_key()

    numbers = pubkey.public_numbers()
    return numbers.n, numbers.e

In [28]:
N0, e0 = read_modulus_from_pem("clef0_pub.pem")
N1, e1 = read_modulus_from_pem("clef1_pub.pem")
N2, e2 = read_modulus_from_pem("clef2_pub.pem")

print("e0, e1, e2 =", e0, e1, e2)
print("N0 =", N0)
print("N1 =", N1)
print("N2 =", N2)

ValueError: error parsing asn1 value: ParseError { kind: ShortData { needed: 1 } }

In [ ]:
m = recover_message_from_broadcast(c0, c1, c2, N0, N1, N2)
print("m =", m)


In [ ]:
m_bytes = m.to_bytes((m.bit_length() + 7) // 8, "big")
print(m_bytes)

try:
    print(m_bytes.decode("utf-8"))
except:
    print("Décodage impossible (pas UTF-8).")
